In [8]:
!pip install -U sentence-transformers

In [17]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
from sentence_transformers import losses
from sentence_transformers import datasets
from sentence_transformers import InputExample
import transformers
print(transformers.__version__)

4.11.0


In [10]:
word_emb = models.Transformer('sentence-transformers/msmarco-distilbert-dot-v5')
pooling = models.Pooling(word_emb.get_word_embedding_dimension())
model = SentenceTransformer(modules = [word_emb, pooling])

We use <b>MultipleNegativesRankingLoss</b> as our loss function for fine-tuning as we only have positive pairs in our dataset <br>(e.g. question->answer pairs)

In [18]:
train_df = pd.read_csv("../0.Datasets/train_test_split/train.csv")
train_df.head()

,Unnamed: 0,qid,docid,question,answer
0,13558,7658,171072,Are RSUs ever taxed as long term capital gains?,"I don't know the legal framework for RSUs, so ..."
1,7820,4290,403701,Are there special exceptions to the rule that ...,This is really an extended comment on the last...
2,15356,9298,481203,Info about managment compansation schemes in p...,Converting the comment from @MD-Tech into answ...
3,12640,7058,416007,Using stop-loss as risk management: Is it safe?,A stop-loss order becomes a market order when ...
4,11509,6319,331722,Making a big purchase over $2500. I have the m...,There are a couple of things to consider here ...


In [20]:
fine_tune_examples = []
for _,row in train_df.iterrows():
    question = row["question"]
    answer = row["answer"]
    fine_tune_examples.append(InputExample(texts=[question,answer]))

In [26]:
train_dataloader = datasets.NoDuplicatesDataLoader(fine_tune_examples, batch_size=128)

In [27]:
train_loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    show_progress_bar=True
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/106 [00:00<?, ?it/s]

C:\Users\yiehy\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [ ]:
# We did this on google colab. Please uncomment and change directory accordingly

# model.save('./finetuned-distilbert-1epoch')
# !zip -r ./finetuned-distilbert-1epoch.zip ./finetuned-distilbert-1epoch
# from google.colab import files
# files.download("./finetuned-distilbert-1epoch.zip")